# VAE CNN Autoencoder GPU Test

Notebook for testing class `VaeCnnAutoencoder` features namely:

1. Initialization with parameters
2. Training with `fit` method using the GPU

In this example, we will be loading images from `input_img_dir` for testing. Images will be 50 width and 50 height in dimensionality.

In [ ]:
# Import necessary libraries and path relative to project
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import cv2
import glob
import numpy as np
import math
from matplotlib import pyplot as plt

%matplotlib inline

import sys
import os

sys.path.append(os.path.join(os.path.abspath(''), '../pyno/lib'))

from vae_cnn_autoencoder import VaeCnnAutoencoder
from abstract_dataset import AbstractDataset

if not torch.cuda.is_available():
    raise Exception("CUDA not available")
    
# CUDA related information
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION', )
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

## Initialization Parameters

In [ ]:
input_img_dir = "/home/ralampay/Pictures/niece"
output_model_file = "/home/ralampay/Desktop/cnn-model.pth"
channel_maps = [3, 16, 4]
h_activation = 'relu'
o_activation = 'sigmoid'
device = torch.device('cuda:0')
optimizer_type = 'adam'
scale = 2
padding = 1
kernel_size = 3
num_channels = 3
img_width = 252
img_height = 252
chunk_size = 100

# (W−F+2P)/S+1 should be an integer

In [ ]:
# Initialize the autoencoder
model = VaeCnnAutoencoder(
            scale=scale,
            channel_maps=channel_maps,
            padding=padding,
            kernel_size=kernel_size,
            num_channels=num_channels,
            img_width=img_width,
            img_height=img_height,
            h_activation=h_activation, 
            o_activation=o_activation, 
            device=device,  
        )

print(model)

## Loading the Dataset

In [ ]:
def load_image_tensors(input_dir, img_width, img_height):
    images = []
    
    ext = ['png', 'jpg', 'gif']    # Add image formats here

    files = []
    [files.extend(glob.glob(input_img_dir + '/*.' + e)) for e in ext]
    
    dim = (img_width, img_height)
    
    images = np.array([cv2.resize(cv2.imread(file), dim) for file in files])
    
    images = images / 255
    
    x = []
    
    for img in images:
#         plt.imshow(img)
#         plt.show()
#         print(img.shape)
        result = img.transpose((2, 0, 1))
        x.append(result)
    
    return torch.tensor(x).float()

x = load_image_tensors(input_img_dir, img_width, img_height)
x = x.to(device)

## Training with `fit` method

In [ ]:
batch_size = 1
epochs = 1000
lr = 0.001
optimizer_type = "adam"

# Reset errors to empty list
model.errs = []

data        = AbstractDataset(x)
dataloader  = DataLoader(dataset=data, batch_size=batch_size, shuffle=True, drop_last=False)

if optimizer_type == "adam":
    model.optimizer = optim.Adam(model.parameters(), lr=lr)
else:
    raise Exception("Invalid optimizer_type: {}".format(optimizer_type))

num_iterations = len(x) / batch_size

for epoch in range(epochs):
    curr_loss = 0 

    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(model.device), labels.to(model.device)
        model.optimizer.zero_grad()

        output, mu, log_var = model.forward(inputs)
        
         # The loss is the BCE loss combined with the KL divergence to ensure the distribution is learnt
        kl_divergence = 0.5 * torch.sum(-1 - log_var + mu.pow(2) + log_var.exp())
        loss = F.binary_cross_entropy(output, inputs, size_average=False) + kl_divergence
        
#         print(output.shape)
#         print(labels.shape)

        loss = model.criterion(output, labels)

        curr_loss += loss
        loss.backward()
        model.optimizer.step()
        
        # display the images
        raw_output = output.detach().cpu().numpy()
        
#         for o in raw_output:
#             result = o.transpose((1, 2, 0))
#             plt.imshow(result)
#             plt.show()

    curr_loss = curr_loss / num_iterations

    print("Epoch: %i\tLoss: %0.5f" % (epoch + 1, curr_loss.item()))

    model.errs.append(curr_loss.detach())

## Checking

In [ ]:
# y = model.forward(x).detach().cpu().numpy()
# print(y.shape)

# for img in y:
#     result = img.transpose((1, 2, 0))
#     print(result)
#     plt.imshow(result)
#     plt.show()
from torchvision import transforms
import torch.nn.functional as F

# Create noise
noise = torch.randn(100, model.z_dim).to(device)
y = model.deconv(noise).detach().cpu().numpy()
dir_output_images = "/home/ralampay/Pictures/niece-random-vae"
counter = 0
for img in y:
    result = img.transpose((1, 2, 0))
    result = result * 255
    file_to_write = "{}/{}.jpg".format(dir_output_images, str(counter))
    cv2.imwrite(file_to_write, result)
    counter = counter + 1